<a href="https://colab.research.google.com/github/rodrigoep11/2023/blob/main/Intro_ML_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code to load the dataset

In [ ]:
!pip install -U "datasets==3.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
!nvidia-smi

Fri Jun 27 06:58:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
### YOU MUST NOT CHANGE THIS CELL! ###

from datasets import load_dataset

full_dataset = load_dataset("skeskinen/TinyStories-GPT4", split="train")
full_dataset = full_dataset.remove_columns([c for c in full_dataset.column_names if c not in ["story", "features"]])
assert len(full_dataset) == 2745100

splits = full_dataset.train_test_split(test_size=10000, seed=42, shuffle=True)

train_dataset = splits["train"]
test_dataset  = splits["test"]

assert len(train_dataset) == 2735100
assert len(test_dataset)  == 10000

assert train_dataset[0]["story"][:33] == "One day, a little girl named Lily"
assert train_dataset[0]["features"] == ["Dialogue", "Conflict"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/554 [00:00<?, ?B/s]

(…)-00000-of-00008-c63ccd5d5290f4a1.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00001-of-00008-478199d8ac044910.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00002-of-00008-9b868f59be94d815.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

...

(…)-00004-of-00008-5f8ac0bb66de5834.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00005-of-00008-e8c22c3e776b87dd.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00006-of-00008-941f57106aca3340.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00007-of-00008-771d8aa2d5ce5c95.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2745100 [00:00<?, ? examples/s]

In [ ]:
# Here we print the first example of the train dataset

from pprint import pprint
pprint(train_dataset[0])

{'features': ['Dialogue', 'Conflict'],
 'story': 'One day, a little girl named Lily went for a walk with her mom. '
          'They saw a big cliff near the water. Lily was fearful and did not '
          "want to go near the cliff. She held her mom's hand tight.\n"
          '"Mom, I am scared," Lily said. Her mom looked at her and smiled. '
          '"It\'s okay to be scared, but I will tell you a secret. When I am '
          'scared, I take a deep breath and count to three. Then, I feel '
          'better."\n'
          'Lily tried what her mom said. She took a deep breath and counted to '
          'three. She felt better and was not as scared. Then, a big bird flew '
          'by and almost hit Lily. She was scared again, but her mom was there '
          'to help her.\n'
          '"Remember what I told you, Lily. Take a deep breath and count to '
          'three," her mom said. Lily did it again, and she felt better. They '
          'walked away from the cliff, and Lily wa

#A first warmup : Using a pretrained T5 Transformers model
## 1) Preprocessing
From the first lines, the dataset is already preprocessed. Rq : there are some stories without any label ! Still, we can also associate the null vector to those inputs so we are fine.

Eg :
<pre><code>print(train_dataset[0]["features"])
 # Out: ['Dialogue', 'Conflict'] </code></pre>

 ## 2) Tokenization
 As we will use a T5 architecture (https://huggingface.co/docs/transformers/model_doc/t5, why ?), we can simply use the tokenization given by HuggingFace (for a review, see https://huggingface.co/docs/transformers/v4.52.3/en/fast_tokenizers. how does it works ? There are no T5tokenizersfast ?.
 Something that keep in consideration is that the order of the features should not matter with the generation of the story, as it was never specified that there was a "main theme" and a "second theme". Thus, we should first order the features.

 Parameters :

 Padding adds a special padding token to ensure all sequences have the same length. Set padding=True to pad the sequences to the longest sequence length in the batch.

 Truncation removes tokens from a sequence to ensure it doesn’t exceed the maximum length. Set truncation=True to truncate a sequence to the maximum length accepted by the model. (this should be check as it can be bad I guess ?)

 return_tensors parameter determines the format in which the tokenized output is returned. This affects how the data can be used in subsequent steps, especially when preparing inputs for a model. pt means pytorch.

## 3) Import of the pre-trained model and finetuning





In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(dataset):
    input_text = " | ".join(sorted(dataset["features"])) # we want to sort the features, so that the two story with features "spooky,love" and "love,spooky" are equivalent
    target_text = dataset["story"]

    model_input = tokenizer(
        input_text,
        max_length=128,
        padding="max_length",
        truncation=True
    )
    with tokenizer.as_target_tokenizer(): #different tokenization procedure for stories bc ???
        labels = tokenizer(
            target_text,
            max_length=512,
            padding="max_length",
            truncation=True
        )

    model_input["labels"] = labels["input_ids"]
    return model_input
small_train = train_dataset.select(range(5000))
tokenized_train = small_train.map(preprocess, remove_columns=train_dataset.column_names) #map is a way to apply this function to data one by one and to store everything in the right way
small_test = test_dataset.select(range(2)) # Using a smaller test set for quicker evaluation
tokenized_test = small_test.map(preprocess, remove_columns=test_dataset.column_names)
print(tokenized_train)
print(tokenized_test)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
import torch

model = T5ForConditionalGeneration.from_pretrained("t5-small") #can we use other things ?
model.to("cuda")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",  # or "steps"
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),  # Use half-precision if GPU sup
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,  # Added the evaluation dataset
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-9-4222831950.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rodrigoep-ortiz (rodrigoep-ortiz-universit-di-bologna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.286000,1.395074
2,1.170100,1.362405
3,1.159800,1.353643


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1875, training_loss=1.4199898681640626, metrics={'train_runtime': 490.3344, 'train_samples_per_second': 30.591, 'train_steps_per_second': 3.824, 'total_flos': 507531755520000.0, 'train_loss': 1.4199898681640626, 'epoch': 3.0})

In [ ]:
def generate_story(tags):
    input_text = " | ".join(sorted(tags))
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    output_ids = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Try it on test examples
for example in test_dataset.select(range(5)):
    print("TAGS:", example["features"])
    print("REAL STORY:", example["story"])
    print("GENERATED:", generate_story(example["features"]))
    print("="*80)

TAGS: ['Dialogue']
REAL STORY: Once upon a time, there was a little girl, named Lucy. She was three years old and loved to be tidy. One day, she was playing in her room and noticed her rug was a little messy. Lucy wanted to make it neat, so she went to her mum and dad and asked for their help.
Her mum smiled and said, "That's a great idea! Let's go get a broom and dustpan and tidy up together." Lucy happily agreed and they got to work.
Once they had finished, Lucy's room was tidy and the rug looked perfect. Her mum looked content and said, "Let's hang it up so that it stays neat". Lucy was so proud of her rug and she suggested they hang it up on the wall.
Her mum smiled, "Good idea! You can use this command strip to make sure it's secure." Soon, the rug was hanging and Lucy felt very proud. She was pleased with the idea of having a tidy rug in her room.
GENERATED: Once upon a time, there was a little girl named Tim. Tim loved to play with his friends. Tim loved to play with his friends

In [ ]:
!pip show transformers

# Track 2 evaluation

The following is a sample implementation of the average cross-entropy loss. As input, you should give a list of lists of probabilities assigned to the tokens (there is one list for each story). You should include the probability assigned to the end-of-sequence token, because part of the task is to predict when the story ends.

In [ ]:
import numpy as np

def avg_cross_entropy(token_probs):
    """
    Average over all stories of the sum of -log p for every ground-truth token.

    token_probs : list[list[float]]
        Probabilities assigned to the correct token at each position.
    """
    return np.average([-np.log(story_token_probs).sum() for story_token_probs in token_probs])

The following is an example of how to call `avg_cross_entropy()`.

In [ ]:
token_probs = [
    [0.9, 0.8, 0.7],    # story 1
    [0.6, 0.5]          # story 2
]

print(avg_cross_entropy(token_probs))

0.9445759076183522
